This script has been modified for 5 seconds video duration on April 29th  2021

In [1]:
import tensorflow as tf
from moviepy.editor import VideoFileClip
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import cv2
import os
import tqdm
import heapq
import datetime
import glob
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from absl import logging
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
logging.set_verbosity(logging.ERROR)
# Some modules to help with reading the UCF101 dataset.
import random
import re
import os
import tempfile
import ssl
# Some modules to display an animation using imageio.
import imageio
from IPython import display
from youtube_dl import YoutubeDL
from urllib import request

In [2]:
import tensorflow as tf
from moviepy.editor import VideoFileClip
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import cv2
import os
import tqdm
import heapq
import datetime
import random
import re
import os
import tempfile
import ssl


In [3]:
PATH=os.getcwd()
BASE_PATH = PATH+'/newdataset/'
VIDEOS_PATH = os.path.join(BASE_PATH, '**','*.mp4')
FILE_SELECTOR = "**/*.mp4"
SEQUENCE_LENGTH = 150

In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

2 Physical GPUs, 2 Logical GPUs


In [5]:
strategy = tf.distribute.MultiWorkerMirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/device:GPU:0', '/device:GPU:1')
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:GPU:0', '/device:GPU:1'), communication = CommunicationImplementation.AUTO


In [6]:
#Code to see the videos that are loaded in Gif format. pass in the output of generator. The np.array only

def to_gif(images):
    #Code to see the videos that are loaded in Gif format. pass in the output of generator. The np.array only
    converted_images =np.clip(images.numpy()[0]* 255, 0, 255).astype(np.uint8)
    gifnum=len([f for f in glob.glob('/home/adewopva/Codes/'+"**/*.gif", recursive=True)])+1
    imageio.mimsave('./animation'+str(gifnum)+'.gif', converted_images, fps=25)
    #IPython.display.Image('animation.gif')
    return embed.embed_file('./animation'+str(gifnum)+'.gif')
    ![SegmentLocal](191px-Seven_segment_display-animation.gif "segment")
    

# to_gif(***sample_video)

In [7]:
#Function for predicting Test video
def video_test(video_paths):

    def crop_center_square(frame):
        (y, x) = frame.shape[0:2]
        min_dim = min(y, x)
        start_x = x // 2 - min_dim // 2
        start_y = y // 2 - min_dim // 2
        return frame[start_y:start_y + min_dim, start_x:start_x
                     + min_dim]

    max_frames = 150
    cap = cv2.VideoCapture(video_paths)
    #frames = []
    frames = np.zeros(shape=(max_frames, 224, 224, 3))
    i =0
    try:
        while True:
            (ret, frame) = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, (224, 224))
            frame = frame[:, :, [2, 1, 0]]

            frames[i]=frame
            i+=1
            if i==150:
                break
#                 frames.append(frame)

#                 if len(frames) == max_frames:
#                     break
    finally:
        cap.release()
    #yield np.array(frames) / 255.0, Label
    return (tf.constant(frames, dtype=tf.float32)[tf.newaxis, ...])/ 255.0


#Load all the threee saved model

In [8]:
#Load all Saved model
K_I_512D= tf.keras.models.load_model(PATH+'/saved_model/NoDrop_NoDense_5sec_AR_kinetics+ImageNetweightsonly')
K_I_aug_20E= tf.keras.models.load_model(PATH+'/saved_model/augumented_5sec_AR_kinetics+ImageNetweightsonly')
K_3DL= tf.keras.models.load_model(PATH+'/saved_model/5sec_AR_kineticsweightsonly_noflatten_moredense')
# Check its architecture


In [10]:
df

NameError: name 'df' is not defined

#Traditional prediction with output of result
result=new_model.predict(clip)
print(result)
result=result.argmax(1)
result

In [ ]:
#All embedded script to split, get duration, crop., preprocess videos and make predictions
import math
from IPython.display import Video
def all_embed(videopath):
    labels=['on_feet', 'active', 'rest', 'escape', 'crawling']
    
    def get_duration(path):      #Get duration of clips function
        clip= cv2.VideoCapture(path)
        totalframes = int(clip.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = int(clip.get(cv2.CAP_PROP_FPS))
        rawduration= int(totalframes / fps)
        #clip = VideoFileClip(path)
        #rawduration= clip.duration
        return rawduration
    
    def get_dir(filename): #Get the path name for any input
        Fname=filename.split('/')[-1]
        file=Fname.split('.')
        dir=file[0].split('/')[-1]
        return dir
    def to_gif(images):
        #Code to see the videos that are loaded in Gif format. pass in the output of generator. The np.array only
        converted_images =np.clip(images.numpy()[0] * 255, 0, 255).astype(np.uint8)
        gifnum=len([f for f in glob.glob('/home/adewopva/Codes/'+"**/*.gif", recursive=True)])+1
        imageio.mimsave('./animation'+str(gifnum)+'.gif', converted_images, fps=25)
        #IPython.display.Image('animation.gif')
        return embed.embed_file('./animation'+str(gifnum)+'.gif')
        ![SegmentLocal](191px-Seven_segment_display-animation.gif "segment")
    def video_test(video_paths):

        def crop_center_square(frame):
            (y, x) = frame.shape[0:2]
            min_dim = min(y, x)
            start_x = x // 2 - min_dim // 2
            start_y = y // 2 - min_dim // 2
            return frame[start_y:start_y + min_dim, start_x:start_x
                         + min_dim]
        
        cap = cv2.VideoCapture(video_paths)
        max_frames =150 #int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        #frames = []
        frames = np.zeros(shape=(max_frames, 224, 224, 3))
        i =0
        try:
            while True:
                (ret, frame) = cap.read()
                if not ret:
                    break
                frame = crop_center_square(frame)
                frame = cv2.resize(frame, (224, 224))
                frame = frame[:, :, [2, 1, 0]]

                frames[i]=frame
                i+=1
                if i==150: #max_frames:
                    break
        #                 frames.append(frame)

        #                 if len(frames) == max_frames:
        #                     break
        finally:
            cap.release()
        #yield np.array(frames) / 255.0, Label
        return (tf.constant(frames, dtype=tf.float32)[tf.newaxis, ...])/ 255.0


    def split_videos(videopath):#Split videos
        rawduration=math.ceil(get_duration(videopath))
        divisorrate=math.ceil(rawduration/5)    #Get the divsion rate to know how many number of iterations will be done on the video
        newtime=0
        nextstart=5
        print('This Video will be splitted to :', divisorrate,'subvideos')
        #print(rawduration)
        #print(divisorrate)
        for i in range(divisorrate):
            dir=get_dir(videopath)
            !mkdir -p "$dir"
            if rawduration-nextstart>5:

                filename = dir+ "/"+'splitted'+ '-'+str(newtime)+ "-" +str(nextstart)+ ".mp4"
                !ffmpeg -hide_banner -loglevel warning -i "$videopath" -ss $newtime -t 5 -c:v libx264 -crf 30  "$filename"
                #nextstart=newtime+15
            elif rawduration-nextstart<0:
                newtime=(rawduration-5)

                filename = dir+ "/"+'splitted'+ '-'+str(newtime)+ "-" +str(rawduration)+ ".mp4"
                !ffmpeg -hide_banner -loglevel warning -i "$videopath" -ss $newtime -t 5 -c:v libx264 -crf 30  "$filename"
            else:
                newtime=(rawduration-5)-(rawduration-nextstart)

                filename = dir+ "/"+'splitted'+ '-'+str(newtime)+ "-" +str(nextstart)+ ".mp4"
                !ffmpeg -hide_banner -loglevel warning -i "$videopath" -ss $newtime -t 5 -c:v libx264 -crf 30  "$filename"
            newtime=nextstart
            nextstart=newtime+5
       
    def iter_predict(videopath):
        if get_duration(videopath)>6:
            BASE_PATH = get_dir(videopath)
            FILE_SELECTOR = "**/*.mp4"
            files = [f for f in glob.glob(BASE_PATH + '/' + FILE_SELECTOR, recursive=True)]
            files = sorted(files, key=lambda x:float(re.findall("(\d+)",x)[0]))
            for file in files:
                prediction_range= (file.split('/splitted-')[-1]).split('.')[0]
                print('\n')
                print('Actions Detected for %s seconds of the input video are:'%(prediction_range))
                clip=video_test(file)
               # print(clip.shape)
                K_I_512D_result= K_I_512D.predict(clip)[0]
                probabilities1 = tf.nn.softmax(K_I_512D_result)
                #print(kin_RGB_k_Iresultresult)
                print('------------------------------------------------------------------') 
                print("The Kinetics and Imagenet 512D model predictions are:")
                for i in np.argsort(probabilities1)[::-1][:3]:
                    print(f"  {labels[i]:}: {probabilities1[i] * 100:5.2f}%")
                    #print('\n')
                print('------------------------------------------------------------------') 
                K_3DL_result= K_3DL.predict(clip)[0]
                probabilities2 = tf.nn.softmax(K_3DL_result)
                #print(kin_RGB_kresultresult)
                print("The Kinetics 3DL model predictions are:")
                for i in np.argsort(probabilities2)[::-1][:3]:
                    print(f"  {labels[i]:}: {probabilities2[i] * 100:5.2f}%")
                print('------------------------------------------------------------------') 
                K_I_aug_20E_result= K_I_aug_20E.predict(clip)[0]
                probabilities3 = tf.nn.softmax(K_I_aug_20E_result)
                #print(kin_RGB_kresultresult)
                print("The Kinetics and Imagenet video_augmented model predictions are:")
                for i in np.argsort(probabilities3)[::-1][:3]:
                    print(f"  {labels[i]:}: {probabilities3[i] * 100:5.2f}%")
                               
        else:
            clip=video_test(videopath)
            #print(clip.shape)
            K_I_512D_result= K_I_512D.predict(clip)[0]
            probabilities1 = tf.nn.softmax(K_I_512D_result)
            #print(kin_RGB_k_Iresultresult)
            print('------------------------------------------------------------------') 
            print("The Kinetics and Imagenet 512D model predictions are:")
            for i in np.argsort(probabilities1)[::-1][:3]:
                print(f"  {labels[i]:}: {probabilities1[i] * 100:5.2f}%")
            print('------------------------------------------------------------------')  
            K_3DL_result= K_3DL.predict(clip)[0]
            probabilities2 = tf.nn.softmax(K_3DL_result)
            #print(kin_RGB_kresultresult)
            print("The Kinetics 3DL model predictions are:")
            for i in np.argsort(probabilities2)[::-1][:3]:
                print(f"  {labels[i]:}: {probabilities2[i] * 100:5.2f}%")
            print('------------------------------------------------------------------') 
            K_I_aug_20E_result= K_I_aug_20E.predict(clip)[0]
            probabilities3 = tf.nn.softmax(K_I_aug_20E_result)
            #print(kin_RGB_kresultresult)
            print("The Kinetics and Imagenet video_augmented model predictions are:")
            for i in np.argsort(probabilities3)[::-1][:3]:
                print(f"  {labels[i]:}: {probabilities3[i] * 100:5.2f}%")


    if get_duration(videopath)>6: 
        split_videos(videopath)
        iter_predict(videopath)
    else:
        iter_predict(videopath)
        Video(videopath, embed=True, height=240,width=240)
    return to_gif(video_test(videopath))
            

    

In [ ]:
to_gif(video_test('/home/adewopva/Codes/multiple_activities/splitted-85-90.mp4'))

In [ ]:
all_embed('/home/adewopva/Downloads/TestVideosForPhase1(Action Recognition and Prediction)/multiple_activities.mov')

In [ ]:
to_gif(video_test('/home/adewopva/Codes/multiple_activities/splitted-15-20.mp4'))

In [ ]:
!youtube-dl 'https://www.pexels.com/video/baby-girl-crawling-on-the-floor-3196565/'

In [ ]:
all_embed("/home/adewopva/Codes/pexels-sarah-chai-7281356.mp4")

In [ ]:
to_gif(video_test('/home/adewopva/Codes/pexels-sarah-chai-7281356/splitted-0-5.mp4'))

In [ ]:
all_embed("/home/adewopva/Codes/video-371861428.mp4")

In [ ]:
# len([f for f in glob.glob('/home/adewopva/OneDrive/Independent_Study/Dr.Nel'+
#                           'ly/ROUGH Scripts/AR_UCF/Testing_Models/'+"**/*.gif", recursive=True)])+1

In [ ]:
#Recognize Actions using the Kinetics+Imagenet weights Model or Kinetics Model only.
#To use this script the video path must be specified with also the weights/model to use
import sys
import math
from IPython.display import Video
def Allscripts_Recognition(videopath,modelweight):
    labels=['on_feet', 'active', 'rest', 'escape', 'crawling']
    weights=modelweight
    
    def get_duration(path):      #Get duration of clips function
        clip= cv2.VideoCapture(path)
        totalframes = int(clip.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = int(clip.get(cv2.CAP_PROP_FPS))
        rawduration= int(totalframes / fps)
        #clip = VideoFileClip(path)
        #rawduration= clip.duration
        return rawduration
    
    def get_dir(filename): #Get the path name for any input
        Fname=filename.split('/')[-1]
        file=Fname.split('.')
        dir=file[0].split('/')[-1]
        return dir
    def to_gif(images):
        #Code to see the videos that are loaded in Gif format. pass in the output of generator. The np.array only
        converted_images =np.clip(images.numpy()[0] * 255, 0, 255).astype(np.uint8)
        gifnum=len([f for f in glob.glob('/home/adewopva/OneDrive/Independent_Study/Dr.Nelly'+
                '/ROUGH Scripts/AR_UCF/Testing_Models/'+"**/*.gif", recursive=True)])+1
        imageio.mimsave('./animation'+str(gifnum)+'.gif', converted_images, fps=25)
        #IPython.display.Image('animation.gif')
        return embed.embed_file('./animation'+str(gifnum)+'.gif')
        ![SegmentLocal](191px-Seven_segment_display-animation.gif "segment")

    def video_test(video_paths):

        def crop_center_square(frame):
            (y, x) = frame.shape[0:2]
            min_dim = min(y, x)
            start_x = x // 2 - min_dim // 2
            start_y = y // 2 - min_dim // 2
            return frame[start_y:start_y + min_dim, start_x:start_x
                         + min_dim]
        
        cap = cv2.VideoCapture(video_paths)
        max_frames =150 #int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        #frames = []
        frames = np.zeros(shape=(max_frames, 224, 224, 3))
        i =0
        try:
            while True:
                (ret, frame) = cap.read()
                if not ret:
                    break
                frame = crop_center_square(frame)
                frame = cv2.resize(frame, (224, 224))
                frame = frame[:, :, [2, 1, 0]]

                frames[i]=frame
                i+=1
                if i==150: #max_frames:
                    break
        #                 frames.append(frame)

        #                 if len(frames) == max_frames:
        #                     break
        finally:
            cap.release()
        #yield np.array(frames) / 255.0, Label
        return (tf.constant(frames, dtype=tf.float32)[tf.newaxis, ...])/ 255.0



    def split_videos(videopath):#Split videos
        rawduration=math.ceil(get_duration(videopath))
        divisorrate=math.ceil(rawduration/5)    #Get the divsion rate to know how many number of iterations will be done on the video
        newtime=0
        nextstart=5
        print('This Video will be splitted to :', divisorrate,'subvideos')
        #print(rawduration)
        #print(divisorrate)
        for i in range(divisorrate):
            dir=get_dir(videopath)
            !mkdir -p "$dir"
            if rawduration-nextstart>5:

                filename = dir+ "/"+'splitted'+ '-'+str(newtime)+ "-" +str(nextstart)+ ".mp4"
                !ffmpeg -hide_banner -loglevel warning -i "$videopath" -ss $newtime -t 5 -c:v libx264 -crf 30  "$filename"
                #nextstart=newtime+15
            elif rawduration-nextstart<0:
                newtime=(rawduration-5)

                filename = dir+ "/"+'splitted'+ '-'+str(newtime)+ "-" +str(rawduration)+ ".mp4"
                !ffmpeg -hide_banner -loglevel warning -i "$videopath" -ss $newtime -t 5 -c:v libx264 -crf 30  "$filename"
            else:
                newtime=(rawduration-5)-(rawduration-nextstart)

                filename = dir+ "/"+'splitted'+ '-'+str(newtime)+ "-" +str(nextstart)+ ".mp4"
                !ffmpeg -hide_banner -loglevel warning -i "$videopath" -ss $newtime -t 5 -c:v libx264 -crf 30  "$filename"
            newtime=nextstart
            nextstart=newtime+5
       
       
    def iter_predict(videopath):
        if get_duration(videopath)>6:
            BASE_PATH = get_dir(videopath)
            FILE_SELECTOR = "**/*.mp4"
            files = [f for f in glob.glob(BASE_PATH + '/' + FILE_SELECTOR, recursive=True)]
            files = sorted(files, key=lambda x:float(re.findall("(\d+)",x)[0]))
            for file in files:
                prediction_range= (file.split('/splitted-')[-1]).split('.')[0]
                print('\n')
                print('For %s seconds of the input video:'%(prediction_range))
                clip=video_test(file)
               # print(clip.shape)
                if weights=='Kinetics+ImageNet':
                    kin_RGB_k_Iresult= kin_RGB_k_I.predict(clip)[0]
                    probabilities1 = tf.nn.softmax(kin_RGB_k_Iresult)
                    #print(kin_RGB_k_Iresultresult)
                    print('The recognized baby action is : %s' %(labels[kin_RGB_k_Iresult.argmax()].upper()))
                    print('------------------------------------------------------------------') 
                    print("Probability distribution for the 5 classes are:")
                    for i in np.argsort(probabilities1)[::-1][:5]:
                        print(f"  {labels[i]:}: {probabilities1[i] * 100:5.2f}%")
                        #print('\n') 
                elif weights=='Kinetics':
                    kin_RGB_kresult= kin_RGB_k.predict(clip)[0]
                    probabilities2 = tf.nn.softmax(kin_RGB_kresult)
                    print('The recognized baby action is : %s' %(labels[kin_RGB_kresult.argmax()].upper()))
                    #print(probabilities1)
                    print('------------------------------------------------------------------')
                    print("Probability distribution for the 5 classes are:")
                    for i in np.argsort(probabilities2)[::-1][:5]:
                        print(f"  {labels[i]:}: {probabilities2[i] * 100:5.2f}%")
                else:
                    print('The type of model weights used is incorrect')
                    sys.exit()
        else:
            clip=video_test(videopath)
            #print(clip.shape)
            if weights=='Kinetics+ImageNet':
                kin_RGB_k_Iresult= kin_RGB_k_I.predict(clip)[0]
                probabilities1 = tf.nn.softmax(kin_RGB_k_Iresult)
                print('The recognized baby action is : %s' %(labels[kin_RGB_k_Iresult.argmax()].upper()))                #print(probabilities1)
                print('------------------------------------------------------------------')
                print("Probability distribution for the 5 classes are:")
                for i in np.argsort(probabilities1)[::-1][:5]:
                    print(f"  {labels[i]:}: {probabilities1[i] * 100:5.2f}%")
                print('------------------------------------------------------------------')
            elif weights=='Kinetics':
                kin_RGB_kresult= kin_RGB_k.predict(clip)[0]
                probabilities2 = tf.nn.softmax(kin_RGB_kresult)
                print('The recognized baby action is : %s' %(labels[kin_RGB_kresult.argmax()].upper()))
                #print(kin_RGB_kresultresult)
                print('------------------------------------------------------------------')
                print("Probability distribution for the 5 classes are:")
                for i in np.argsort(probabilities2)[::-1][:5]:
                    print(f"  {labels[i]:}: {probabilities2[i] * 100:5.2f}%")
                print('------------------------------------------------------------------')
            else:
                print('The type of model weights used is incorrect')
                sys.exit()
            

    if get_duration(videopath)>6:
        split_videos(videopath)
        iter_predict(videopath)
    else:
        iter_predict(videopath)
        Video(videopath, embed=True, height=240,width=240)
    return to_gif(video_test(videopath))
            

    

In [ ]:
!youtube-dl 'https://www.instagram.com/p/CNtG03dAJSi/' -q
print('Video downloaded')

In [ ]:
videopath='/home/adewopva/Downloads/TestVideosForPhase1(Action Recognition and Prediction)/multiple_activities.mov'
modelweight='Kinetics'
Allscripts_Recognition(videopath,modelweight)

In [ ]:
clip=video_test('/home/adewopva/OneDrive/Independent_Study/Dr.Nelly/ROUGH Scripts/AR_UCF/Testing_Models/Video by ig/splitted-6-16.mp4')

In [ ]:
to_gif(clip)

In [ ]:
all_embed('/home/adewopva/Downloads/TestVideosForPhase1(Action Recognition and Prediction)/multiple_activity_dif_babies.mov')

In [ ]:
labels=['on_feet', 'active', 'rest', 'escape', 'crawling']
clip=video_test('/home/adewopva/OneDrive/Independent_Study/Dr.Nelly/ROUGH Scripts/AR_UCF/Testing_Models/multiple_activities/splitted-0-15.mp4') #FUNCTION TO crop and preprocess video
output1= kin_RGB_k.predict(clip)
print(output1)

kin_RGB_k_Iresult=output1[0]
print(kin_RGB_k_Iresult)
probabilities1 = tf.nn.softmax(kin_RGB_k_Iresult)
#print(kin_RGB_k_Iresultresult)
print("The Kinetics and Imagenet model topmost three predictions are:")
for i in np.argsort(probabilities1)[::-1][:3]:
    print(f"  {labels[i]:}: {probabilities1[i] * 100:5.2f}%")

print('\n')
to_gif(clip)

In [ ]:

def comparemodels(clip):
    labels=['on_feet', 'active', 'rest', 'escape', 'crawling']
    clip=video_test(clip)
    output1= kin_RGB_k_I.predict(clip)
    kin_RGB_k_Iresult=output1[0]
    probabilities1 = tf.nn.softmax(kin_RGB_k_Iresult)
    #print(kin_RGB_k_Iresultresult)
    print("The Kinetics and Imagenet model topmost three predictions are:")
    for i in np.argsort(probabilities1)[::-1][:3]:
        print(f"  {labels[i]:}: {probabilities1[i] * 100:5.2f}%")

    print('\n')


    output2= kin_RGB_k.predict(clip)
    kin_RGB_kresult=output2[0]
    probabilities2 = tf.nn.softmax(kin_RGB_kresult)
    #print(kin_RGB_kresultresult)
    print("The Kinetics only model topmost three predictions are:")
    for i in np.argsort(probabilities2)[::-1][:3]:
        print(f"  {labels[i]:}: {probabilities2[i] * 100:5.2f}%")
        print('\n')

    return to_gif(clip)


In [ ]:
#Amir's test video
comparemodels('/home/adewopva/OneDrive/Independent_Study/Dr.Nelly/ROUGH Scripts/AR_UCF/Testing_Models/Video by sweety_babiez-CNOGgFnA_N_.mp4')

In [ ]:
#Amir's test video
comparemodels('/home/adewopva/Downloads/TestVideosForPhase1(Action Recognition and Prediction)/multiple_activities.mov')

In [ ]:
#Amir's test video
comparemodels('/home/adewopva/Downloads/TestVideosForPhase1(Action Recognition and Prediction)/All_in_crib.mov')

In [ ]:
Download new videos

In [ ]:
!youtube-dl 'https://www.instagram.com/p/CAjFYbWnaxE/'

In [ ]:
#instangram Downloader
!instalooter post 'https://www.instagram.com/p/CGgKBEph3ju/'  ~/Downloads

In [ ]:

#play the downloaded video
from IPython.display import Video

Video('production ID -3676977-388992755.mp4', embed=True, height=240,width=240)

In [ ]:
#Download using youtube dl
!youtube-dl 'https://www.youtube.com/watch?t=4&v=BaW_jenozKc'

In [ ]:
#Links to videos and repo

In [ ]:
2000 babies videos
https://www.pexels.com/search/videos/baby/

In [ ]:
Instagram baby videos
https://www.instagram.com/baby_videos__/?hl=en

In [ ]:
YoutubeDl repo
https://github.com/ytdl-org/youtube-dl